In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.cm as cm
import numpy as np
import astropy.io.ascii as at
import astropy.io.fits as fits
import astropy.units as u
from astropy.time import Time
from astropy.io import fits
from astropy.coordinates import SkyCoord, Distance
from astropy.table import Table,join
# from calc_ruwe import calc_ruwe
# from scipy import stats, spatial

In [2]:
cmap2 = cm.get_cmap("viridis",7)
colors = {"IC_2391": cmap2(0),
         "IC_2602": cmap2(4),
         "NGC_2547": cmap2(3),
         "NGC_2451A": cmap2(2),
         "Collinder_135": cmap2(1)}


shapes= {"IC_2391": "o",
         "IC_2602": "d",
         "NGC_2547": "v",
         "NGC_2451A": "^",
         "Collinder_135": "s"}


In [3]:
cat_dir = os.path.expanduser("~/projects/TESS_young/")

In [4]:
from analyze_cluster_output import id_solar, read_cluster_visual

In [5]:
clusters = ["IC_2391","Collinder_135","NGC_2451A","NGC_2547","IC_2602"]
dates = ["2021-06-22","2021-06-18","2021-06-21","2021-06-21","2021-07-02"]
distances = {"Collinder_135":292,"IC_2602": 149, "NGC_2547":396,
             "IC_2391":148,"NGC_2451A":195}


In [6]:
# i = 0
# cluster = clusters[i]
max_sep = 1e3 * u.AU

for i,cluster in enumerate(clusters):

    hdbscanfile = os.path.expanduser(f"~/Dropbox/EDR3/scats/{cluster}.fits")
    with fits.open(hdbscanfile) as hdu:
        hdbscan = Table(hdu[1].data)
    if hdbscan.masked == False:
        hdbscan = Table(hdbscan, masked=True, copy=False)
    hdbscan.rename_column("GAIADER3_ID","GAIAEDR3_ID")
    hpos = SkyCoord(hdbscan["GAIAEDR3_RA"],hdbscan["GAIAEDR3_DEC"],unit=u.degree)


    dat2 = read_cluster_visual(cluster,dates[i],clean_limit=0,
                               to_plot=False,return_periodcolor=False)
    dpos = SkyCoord(dat2["GAIAEDR3_RA"],dat2["GAIAEDR3_DEC"],unit=u.degree)

    bp_rp = hdbscan["GAIAEDR3_BP"] - hdbscan["GAIAEDR3_RP"]
    bp_rp2 = dat2["GAIAEDR3_BP"] - dat2["GAIAEDR3_RP"]

    solar = id_solar(bp_rp2)

    sep_dist = max_sep.to(u.pc) / (distances[cluster]*u.pc)
    max_ang_dist =  sep_dist.value * 360 / (2*np.pi) * u.degree
    print(max_ang_dist.to(u.arcsec))

    n_solar = len(np.where(solar)[0])
    ndat = len(dat2)
    cand_resolved = np.zeros(ndat,bool)
    cand_loc = np.ones(ndat,int)*-9999

    for j in np.where(solar)[0]:    
        sep = dpos[j].separation(hpos)

        hloc = hdbscan["GAIAEDR3_ID"]==dat2["GAIAEDR3_ID"][j]

        close = (sep < max_ang_dist) & (hdbscan["GAIAEDR3_ID"]!=dat2["GAIAEDR3_ID"][j])

    #     print(len(np.where(close)[0]))
    #     print(sep[close].to(u.arcsec))
        for k in np.where(close)[0]:
            pmra_diff = np.abs(hdbscan["GAIAEDR3_PMRA"][hloc]-hdbscan["GAIAEDR3_PMRA"][k])
            pmdec_diff = np.abs(hdbscan["GAIAEDR3_PMDEC"][hloc]-hdbscan["GAIAEDR3_PMDEC"][k])
            pmra_diff_sigma = np.sqrt(hdbscan["GAIAEDR3_PMRA_ERROR"][hloc]**2 + hdbscan["GAIAEDR3_PMRA_ERROR"][k]**2)
            pmdec_diff_sigma = np.sqrt(hdbscan["GAIAEDR3_PMDEC_ERROR"][hloc]**2 + hdbscan["GAIAEDR3_PMDEC_ERROR"][k]**2)
    #         print(k,pmra_diff,pmdec_diff,pmra_diff_sigma,pmdec_diff_sigma)
            if (pmra_diff < (pmra_diff_sigma*3)) & (pmdec_diff < (pmdec_diff_sigma*3)):
                cand_resolved[j] = True
                cand_loc[j] = k
                print(i,j,k,sep[k].to(u.arcsec))
                print(k,pmra_diff_sigma,pmdec_diff_sigma)
                

    print(cluster,cand_resolved[solar])
# Something is wrong with this because I'm getting a print out for an IC2391 star, but no close targets in the boolean array

224
6.756756756756757 arcsec
IC_2391 [False False False False False False False False]
364
3.4246575342465753 arcsec
Collinder_135 [False False False False False False False False False False False False
 False False False False False]
316
5.128205128205129 arcsec
NGC_2451A [False False False False False False False False False False False]
199
2.525252525252525 arcsec
NGC_2547 [False False False False False False False False False False False False
 False]
395
6.7114093959731544 arcsec
IC_2602 [False False False False False False False False False False False False
 False False]


In [7]:
cand_resolved

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [8]:
hdbscan.dtype

dtype([('H3_ID', '>f8'), ('PS_ID', '>i8'), ('TMASS_ID', '>i8'), ('WISE_ID', '<U28'), ('UNWISE_ID', '<U16'), ('SDSS_ID', '>i8'), ('GALEX_ID', '>i8'), ('DES_ID', '>i8'), ('DECAPS_ID', '>i8'), ('UKIDSS_ID', '>i8'), ('APOPAYNE_ID', '>i8'), ('LAMOST_ID', '>i4'), ('GAIADR1_ID', '>f8'), ('GAIADR2_ID', '>f8'), ('GAIAEDR3_ID', '>i8'), ('TILE_ID', '>f8'), ('RA', '>f8'), ('DEC', '>f8'), ('RA_S', '>f8'), ('DEC_S', '>f8'), ('L', '>f8'), ('B', '>f8'), ('EBV', '>f4'), ('BHB', '>f8'), ('MGIANT', '>f8'), ('RAND', '>f8'), ('GUIDE', '>f8'), ('XFIT_RANK', '>f8'), ('PS_G', '>f4'), ('PS_R', '>f4'), ('PS_I', '>f4'), ('PS_Z', '>f4'), ('PS_Y', '>f4'), ('SDSS_U', '>f4'), ('SDSS_G', '>f4'), ('SDSS_R', '>f4'), ('SDSS_I', '>f4'), ('SDSS_Z', '>f4'), ('TMASS_J', '>f4'), ('TMASS_H', '>f4'), ('TMASS_K', '>f4'), ('WISE_W1', '>f4'), ('WISE_W2', '>f4'), ('WISE_W3', '>f4'), ('WISE_W4', '>f4'), ('UNWISE_W1', '>f4'), ('UNWISE_W2', '>f4'), ('GALEX_NUV', '>f4'), ('GALEX_FUV', '>f4'), ('DES_G', '>f4'), ('DES_R', '>f4'), ('DES_

In [9]:
obs_sources = ["5318059532750974720","5318498688873909632","5317884439832479872","5318500303781947776"]

In [10]:
ic2391_zorro = at.read(os.path.join(cat_dir,"IC2391_zorro_DR2memb_EDR3data.csv"),
                       delimiter=",")

In [12]:
subcat = ic2391_zorro[ic2391_zorro["ID"].mask==False]["ID","RA","Dec","Obs date","Sem","TIC",
                                                      "name","date","wav","rho","theta","B/A",
                                                      "delm","see","rho*see","notes","period"]

i = 0
cluster = clusters[i]

dat2 = read_cluster_visual(cluster,dates[i],clean_limit=0,
                           to_plot=False,return_periodcolor=False)

ic2391_zorro_edr3 = join(subcat,dat2,keys=["TIC"],join_type="left")

224


In [14]:
gemini_det = ic2391_zorro_edr3["rho"].mask==False

In [23]:
# i = 0
# cluster = clusters[i]
max_sep = 1e3 * u.AU

i = 0
cluster = clusters[i]
sep_dist = max_sep.to(u.pc) / (distances[cluster]*u.pc)
max_ang_dist =  sep_dist.value * 360 / (2*np.pi) * u.degree
print(max_ang_dist.to(u.arcsec))


hdbscanfile = os.path.expanduser(f"~/Dropbox/EDR3/scats/{cluster}.fits")
with fits.open(hdbscanfile) as hdu:
    hdbscan = Table(hdu[1].data)
if hdbscan.masked == False:
    hdbscan = Table(hdbscan, masked=True, copy=False)
hdbscan.rename_column("GAIADER3_ID","GAIAEDR3_ID")
hpos = SkyCoord(hdbscan["GAIAEDR3_RA"],hdbscan["GAIAEDR3_DEC"],unit=u.degree)


bp_rp = hdbscan["GAIAEDR3_BP"] - hdbscan["GAIAEDR3_RP"]


for j in np.where(gemini_det)[0]:
    gpos = SkyCoord(ic2391_zorro_edr3["GAIAEDR3_RA"][j],ic2391_zorro_edr3["GAIAEDR3_DEC"][j],unit=u.degree)
    sep = gpos.separation(hpos)
    close = (sep < max_ang_dist) & (hdbscan["GAIAEDR3_ID"]!=ic2391_zorro_edr3["GAIAEDR3_ID"][j])
#     print(np.where(close)[0])
    print(ic2391_zorro_edr3["rho"][j],sep[close].to(u.arcsec))
    print(ic2391_zorro_edr3["ID"][j],(ic2391_zorro_edr3["rho"][j]*(u.arcsec).to(u.radian)*distances[cluster]*u.pc).to(u.AU))

    


6.756756756756757 arcsec
0.416 [5.75759arcsec]
IC2391m083922-5355 61.568 AU
0.044 [2.77718arcsec]
IC2391f084218-5301 6.511999999999999 AU
0.048 []
IC2391b084214-5256 7.104 AU
0.922 []
IC2391b084300-5354 136.45600000000002 AU
